import libraries

In [1]:
import pandas as pd
import numpy as np

data

In [2]:
points_geo = pd.read_csv('df_resolute_CLEANED_4IQR_4PREP_priceperm2IQR_prep.csv')
points_geo.drop(['dist_coast','dist_beach','dist_city','Year','PriceEuro','NAME_LATIN', 'longitude', 'latitude'], axis=1, inplace=True)

In [3]:
points_geo.PricePerM2.describe()

count    76000.000000
mean       931.539874
std        772.606003
min          0.400000
25%        260.000000
50%        781.000000
75%       1389.000000
max       3243.000000
Name: PricePerM2, dtype: float64

In [4]:
points_geo.PricePerM2.hist()

In [5]:
# n_points_geo.info()

pick property type

In [120]:
# n_points_geo = points_geo[points_geo.property_land==1]
# n_points_geo = points_geo[points_geo.property_residential==1]
n_points_geo = points_geo[points_geo.property_business==1]

In [121]:
n_points_geo.PricePerM2.describe()

count    1660.000000
mean      894.201807
std       656.977805
min         5.000000
25%       375.000000
50%       737.000000
75%      1231.750000
max      3240.000000
Name: PricePerM2, dtype: float64

In [108]:
n_points_geo.reset_index(drop=True,inplace=True)
n_points_geo.drop(['Rooms', 'Levels', 'property_business', 'property_land',
                   'property_other', 'property_residential'], axis=1, inplace=True)

/home/lefteris/anaconda3/envs/resolute/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


random forest 

In [109]:
gps_hot = n_points_geo.copy()

In [110]:
gps_predict_label = 'PricePerM2'
gps_test_list = [x for x in gps_hot.columns if x!=gps_predict_label]

In [111]:
# y is the values we want to predict
y = gps_hot[gps_predict_label].values
# OR
# Labels are the values we want to predict
labels = np.array(gps_hot[gps_predict_label])

# Convert to numpy array
X = gps_hot[gps_test_list].values
# OR
# Convert to numpy array
gps_array = np.array(gps_hot[gps_test_list])

split dataset into train and test

In [112]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# # Split the data into training and testing sets
# train_features, test_features, train_labels, test_labels =\
#     train_test_split(gps_array, labels, test_size = 0.20) # , random_state = 42

#OR

# Split the data into training and testing sets
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size = 0.20) # , random_state = 42

In [113]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (1328, 11)
Training Labels Shape: (1328,)
Testing Features Shape: (332, 11)
Testing Labels Shape: (332,)


train

In [114]:
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 100 decision trees
reg = RandomForestRegressor(n_estimators = 100, n_jobs=-1) # , random_state = 42

# Train the model on training data
reg.fit(X_train, y_train) # OR train_features, train_labels

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

predict

In [115]:
# Use the forest's predict method on the test data
y_pred = reg.predict(X_test) #OR test_features

metrics

In [116]:
result = pd.DataFrame(y_pred,y_test).reset_index().rename(columns={'index':'real_cnt',0:'pred_cnt'})
result.sample(5)

real_cnt     pred_cnt
45      667.0   522.670000
308     920.0   756.395000
280    1799.0  1381.180000
61      900.0   982.940000
159    1108.0  1601.537833

In [117]:
from sklearn import metrics

print('error units: Euro PerM2')
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred))) 

error units: Euro PerM2
Mean Absolute Error: 407.8579616863549
Mean Squared Error: 306756.16290498537
Root Mean Squared Error: 553.8557239073957


important features

In [118]:
print('\nRandom Forest: Business\n')

print('Error units: Euro PerM2')
print('Mean Absolute Error:', round(metrics.mean_absolute_error(y_test, y_pred),2),'\n') 

# Get numerical feature importances
importances = list(reg.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(gps_array, round(importance, 3)) for gps_array, importance in zip(gps_test_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
for pair in feature_importances:
    print('Variable: {:20} Importance: {}'.format(*pair))


Random Forest: Business

Error units: Euro PerM2
Mean Absolute Error: 407.86 

Variable: age                  Importance: 0.388
Variable: Area_sqm             Importance: 0.327
Variable: oiki_densi           Importance: 0.085
Variable: pop_densi            Importance: 0.058
Variable: geohash_lat          Importance: 0.053
Variable: geohash_lng          Importance: 0.053
Variable: Bathrooms            Importance: 0.028
Variable: near_city            Importance: 0.005
Variable: near_beach           Importance: 0.003
Variable: near_coast           Importance: 0.001
Variable: Bedrooms             Importance: 0.0


In [119]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline

# Set the style
plt.style.use('fivethirtyeight')

# list of x locations for plotting
x_values = list(range(len(importances)))

# set the size of figure
f, ax = plt.subplots(figsize=(18,5))

# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')

# Tick labels for x axis
plt.xticks(x_values, gps_test_list, rotation='vertical')

# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Business Variable Importances');

check predict

In [18]:
df_to_predict_df = gps_hot[(gps_hot['Bedrooms']==3) &
                               (gps_hot['Area_sqm'] > 100)]

test_value = df_to_predict_df[gps_test_list].values
test_pred_value = df_to_predict_df[[gps_predict_label]].values

In [21]:
pred_value = reg.predict(test_value)

In [22]:
# pd.DataFrame(test_pred_value,pred_value)
df = pd.DataFrame({'pred_cnt':pred_value})
df['actual_cnt'] = test_pred_value

In [40]:
df

pred_cnt  actual_cnt
0     2275.150000      2500.0
1     2901.270000      3000.0
2     2568.780000      2439.0
3     2808.552500      2988.0
4     2753.953000      2625.0
5     1940.930000      1885.0
6     2923.873000      3214.0
7     1806.480000      1885.0
8     1316.550000       887.0
9     1329.320000      1000.0
10    2091.150000      1916.0
11    2194.585000      2000.0
12    2518.276667      2628.0
13    1728.360000      1863.0
14    2137.345000      1858.0
15    2241.580000      2313.0
16    1707.036667      1386.0
17    2025.230000      1818.0
18    2922.620000      2984.0
19    2214.810000      1922.0
20    1677.260000      1502.0
21    2661.810000      2703.0
22    2396.023333      2523.0
23    1615.410000      1588.0
24    2247.890000      2427.0
25    2818.060000      3185.0
26    1498.930000       900.0
27    2753.953000      3042.0
28    2322.400000      2826.0
29    2127.117500      2000.0
...           ...         ...
6871   871.700000       963.0
6872   766.283333       600.0
6873   955.225000       917.0
6874   955.225000       917.0
6875   604.955000       542.0
6876   599.875000       464.0
6877  1272.950000      1091.0
6878  1060.250000       818.0
6879  1727.000000      1816.0
6880  1058.940000      1019.0
6881   799.180000       598.0
6882   987.810000       717.0
6883   927.710000       964.0
6884   714.580000       738.0
6885  1861.860000      2125.0
6886   740.410000       567.0
6887  2157.682500      1250.0
6888   564.950000       421.0
6889   341.420000       215.0
6890  1062.920000      1563.0
6891   877.070000       701.0
6892   912.955000       792.0
6893   937.955000       943.0
6894  1360.910000      1429.0
6895  1462.630000      1487.0
6896   873.620000       702.0
6897   910.490000       858.0
6898   534.110000       417.0
6899   636.320000       593.0
6900   327.450000       212.0

[6901 rows x 2 columns]